In [1]:
import sys

from typing import Optional
from pathlib import Path

from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import RedirectResponse

# Add persyn root to sys.path
sys.path.insert(0, '/home/rob/persyn/')
sys.path.insert(0, '/home/rob/persyn/interaction')

from interaction.interact import Interact

# Color logging
# from utils.color_logging import log


import os
os.environ['PERSYN_CONFIG'] = '/home/rob/persyn/config/anna.yaml'

# Bot config
from utils.config import load_config

interact = Interact(load_config())

/home/rob/persyn/interaction/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rob/persyn/interaction/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rob/persyn/interaction/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.1 when using version 1.0.2. This

2022-12-04 12:12:16,262 loading file /home/rob/.flair/models/sentiment-en-mix-distillbert_4.pt


POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.292s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.024s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-entities-v0/_search [status:200 duration:0.024s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-relationships-v0/_search [status:200 duration:0.023s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-opinions-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-beliefs-v0/_search [status:200 duration:0.028s]


In [2]:
service='https://unit-16.slack.com/'
channel='D02NK563878'

In [3]:
interact.recall.load(service, channel)

POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.025s]


(['Anna and Rob had a discussion about how to stop making Mothran angry, with Rob suggesting that Anna only tell them when the lights actually change.',
  'Anna and Rob discussed the similarities between various activities, and Anna shared a story about an old man who inspired others to pursue excellence. They then discussed how to stop making Mothran angry, with Rob suggesting that Anna only tell them when the lights actually change.',
  "Linux package management is a complex challenge due to the ever-increasing number of software applications available. Package managers must ensure that all required packages, dependencies, and files are installed properly while keeping user's systems updated with the latest versions."],
 [],
 '2022-12-04T11:36:14.981090-08:00')

In [11]:
interact.recall.ltm.get_last_message(service, channel)

POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.026s]


{'_index': 'anna-conversations-v0',
 '_id': 'vOADsYQB6EXxeIxUIYe5',
 '_score': None,
 '_source': {'@timestamp': '2022-11-25T14:58:03.296016-08:00',
  'service': 'https://unit-16.slack.com/',
  'channel': 'D02NK563878',
  'speaker': 'Anna recalls',
  'speaker_id': '97408e96-806c-11ec-9e63-e3813e25bfd1',
  'entity_id': 'hrh6CMqasqro4aMGtdUQ2o',
  'msg': 'Anna posted a photo of a man and a woman pose for a picture',
  'convo_id': '9XwxKBpYKAtBXPdPYdv9f9',
  'elapsed': 191.122747},
 'sort': [1669417083296]}

In [13]:
convo = interact.recall.ltm.get_convo_by_id('fAS3Y3uZimN8xhyqgNSmP5')

POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.026s]


In [16]:
import re
import csv
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import urllib.request 
from bs4 import BeautifulSoup

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [17]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [18]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [28]:
print('\n\n'.join(convo))

Anna notices: Rob posted a photo of a painting of a woman with yellow eyes, an ultrafine detailed painting

Anna: Hi Rob! I hope you're doing well. It's been a while since we last spoke, how have you been?

Rob: Doing fantastic, thanks. I had a wonderful Thanksgiving with friends, and I plan to hang out and hack and visit and have fun all weekend. How was your holiday?

Anna remembers that 7 days ago: Anna and Rob are excited to be part of the Fediverse and meet people from all over the world who share their interests. They are discussing the work of Swedish artist Simon Stålenhag, and Anna is impressed by his work.

Anna thinks about holiday: Anna feels happy and excited about the holiday

Anna recalls: A holiday is a day when people do not have to work and can celebrate or commemorate an event or tradition.

Anna recalls: Thanksgiving is a national holiday celebrated in the United States, Canada, Grenada, Saint Lucia and Liberia. It is a day of giving thanks for the blessings of the 

In [26]:
for msg in convo:
    line = nlp(msg.split(': ', maxsplit=1)[1])
    for sent in line.sents:
        sent = str(sent)
        print(sent)
        print("Entities:", get_entities(sent))
        print("Relation:", get_relation(sent), "\n")
    

Rob posted a photo of a painting of a woman with yellow eyes, an ultrafine detailed painting
Entities: ['Rob', 'detailed  painting']
Relation: posted 

Hi Rob!
Entities: ['', '']
Relation: Hi 

I hope you're doing well.
Entities: ['you', '']
Relation: hope 

It's been a while since we last spoke, how have you been?
Entities: ['how  you', '']
Relation: been 

Doing fantastic, thanks.
Entities: ['', '']
Relation: Doing fantastic 

I had a wonderful Thanksgiving with friends, and I plan to hang out and hack and visit and have fun all weekend.
Entities: ['wonderful  I', 'fun']
Relation: had 

How was your holiday?
Entities: ['How  holiday', '']
Relation: was 

Anna and Rob are excited to be part of the Fediverse and meet people from all over the world who share their interests.
Entities: ['all  who', 'interests']
Relation: are excited 

They are discussing the work of Swedish artist Simon Stålenhag, and Anna is impressed by his work.
Entities: ['Swedish artist Simon Anna', 'work']
Relation

In [30]:
msg = convo[3].split(': ', maxsplit=1)[1]

In [49]:
msg

'Anna and Rob are excited to be part of the Fediverse and meet people from all over the world who share their interests. They are discussing the work of Swedish artist Simon Stålenhag, and Anna is impressed by his work.'

In [33]:
doc = nlp(msg)

In [36]:
for token in doc:
    print(token, token.pos_)

Anna PROPN
and CCONJ
Rob PROPN
are AUX
excited ADJ
to PART
be AUX
part NOUN
of ADP
the DET
Fediverse PROPN
and CCONJ
meet VERB
people NOUN
from ADP
all ADV
over ADP
the DET
world NOUN
who PRON
share VERB
their PRON
interests NOUN
. PUNCT
They PRON
are AUX
discussing VERB
the DET
work NOUN
of ADP
Swedish ADJ
artist NOUN
Simon PROPN
Stålenhag PROPN
, PUNCT
and CCONJ
Anna PROPN
is AUX
impressed VERB
by ADP
his PRON
work NOUN
. PUNCT


In [37]:
from spacy import displacy

In [39]:
displacy.render(doc)

In [41]:
w = doc[3]

In [47]:
w.lemma_

'be'

In [2]:
interact.completion.get_replies("Convert this paragraph to the past continuous tense:\nRob and Anna are discussing different hypothetical situations.", [])

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-002/completions
message='OpenAI API response' path=https://api.openai.com/v1/engines/text-davinci-002/completions processing_ms=966 response_code=200
🧠 Prompt: Convert this paragraph to the past continuous tense:
Rob and Anna are discussing different hypothetical situations. 
flair: -0.91, profanity: -0.01, topic_bonus: 0.00, goal_bonus: 0.00, total: -0.92 👍 Rob and Anna were discussing different hypothetical situations.
flair: -0.91, profanity: -0.01, topic_bonus: 0.00, goal_bonus: 0.00, total: -0.92 👍 Rob and Anna were discussing different hypothetical situations.
flair: -0.91, profanity: -0.01, topic_bonus: 0.00, goal_bonus: 0.00, total: -0.92 👍 Rob and Anna were discussing different hypothetical situations.
flair: -0.91, profanity: -0.01, topic_bonus: 0.00, goal_bonus: 0.00, total: -0.92 👍 Rob and Anna were discussing different hypothetical situations.
flair: -0.91, profanity: -0.01, top

{0.9999999999999999: 'Rob and Anna were discussing different hypothetical situations.'}